In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} PWA101: Amplitude Analysis with Python basics
TR-999
^^^
+++
✅&nbsp;[ComPWA/RUB-EP1-AG#93](https://github.com/ComPWA/RUB-EP1-AG/issues/93), [compwa.github.io#217](https://github.com/ComPWA/compwa.github.io/pull/217)
:::
::::

# Amplitude Analysis 101

In [ ]:
%pip install -q "phasespace[fromdecay]"==1.9.0 gdown==4.7.1 iminuit==2.25.2 matplotlib==3.7.3 numpy==1.24.4 pandas==2.2.1 particle==0.23.0 scipy==1.10.1 vector==1.1.1.post1

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import vector
from matplotlib import gridspec
from vector.backends.numpy import MomentumNumpy4D

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

This document serves as an introduction to Amplitude Analysis / Partial Wave Analysis (PWA) by demonstrating its application to a specific reaction channel and amplitude model below, utilizing basic Python programming (e.g. `numpy`, `scipy`, ...). Before advancing to the complexities of symbolic expressions (computations) with `sympy`, this tutorial aims to equip readers with a basic understanding of PWA methodologies and full workflow in the context of hadron physics through a practical, hands-on example.

## Amplitude model

<!-- cspell:ignore Mathieu -->
This amplitude model is adapted from the [Lecture 11 in STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803/contributions/21223/) by Vincent Mathieu.

The (photo-production) reaction is $ \gamma p \to \eta \pi^0 p$, it is described by a amplitude model with three possible resonances: $a_2$, $\Delta^+$, and $N^*$. 

```{image} https://github.com/ComPWA/compwa-org/assets/17490173/ec6bf191-bd5f-43b0-a6cb-da470b071630
:width: 100%
```

$$
\begin{eqnarray}
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \times s^{0.5+0.9u_3}  \nonumber \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} \times s^{0.5+0.9t_1}  \nonumber \\
A^{31} &=& \frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \times s^{1.08+0.2t_2} 
\end{eqnarray}
$$ (full_model_label)

where $s, t, u$ are the Mandelstam variables $s_{ij}=(p_i+p_j)^2$, $t_i=(p_a-p_i)^2$, and $u_i=(p_b-p_i)^2$.

The original full amplitude model from the [Lecture 11 in STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803/contributions/21223/) is shown in equation {eq}`full_model_label`.

*In this report, only the Breit-Wigner and Spherical harmonics terms are kept for doing PWA eventually (The exponential mandelstam variable term is abandoned), as shown in equation {eq}`BW_SH_label`.

$$
\begin{eqnarray}
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}}  \\
\
A^{31} &=& \frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} 
\end{eqnarray}
$$ (BW_SH_label)

with  intensity $I$ and amplitude $A$:

$$
\begin{eqnarray}
I &=& |A|^2 \nonumber \\
A &=& A^{12} + A^{23} + A^{31}  \\
\end{eqnarray}
$$ (123_label)

where $\quad 1 \equiv \eta ; \quad  2 \equiv \pi^0 ; \quad 3 \equiv p$

:::{note}
The ultimate choice of the amplitude model (equations (2) and (3)) for PWA in this tutorial consists of three resonances, and each of them are formed by two terms: Breit-Wigner with Spherical harmonics ($l = 2, 1, 0$).
:::

:::{important}
The spin of $\eta$ meson and $\pi^0$ meson are all 0. But the spin of proton is not 0, it is spin-$\frac{1}{2}$.

In this amplitude model **spin** of baryon is simplified (not realistic):
$\eta$, $\pi^0$ and $p$ are all treated as spin-0 particles.

This means that total intrinsic spin $s$ is ignored in this model,
the total angular momentum 
$J$ of the system or any subsystems within this model will solely depend on the orbital angular momentum 
$L$, characterized by quantum number $l$.
And this simplifies the use of spherical harmonics $Y_l^m(\theta,\phi)$,
since only the orbital angular momentum component is involved, and thus the combination of contribution is not considered (e.g. Clebsch-Gordan Coefficients).

In our case: 
- $A^{12}$ amplitude represents a d-wave interaction, as indicated by $l=2$.
    - The possible $m$ values are $−2,−1,0,1,2$. Each of these values corresponds to different orientations of the d-wave. The wave type is solely determined by $l$ and all these $m$ values still describe d-wave characteristics.
- $A^{23}$ amplitude represents a p-wave interaction, as indicated by $l=1$.
    - The possible $m$ values are $−1,0,1$. Each of these values corresponds to different orientations of the p-wave. Similarly, these values are all p-wave orientations.
- $A^{31}$ amplitude represents a s-wave interaction, as indicated by $l=0$.
    - The only possible $m$ value is 0, which is consistent with the spherical symmetry of s-waves.
:::

## Number of events

The number of events for phase generation and data generation can be changed in this section.

In [ ]:
phsp_events = 1_000_000
data_events = 500_000

## Phase space generation

In this section, the phase space of this reaction is generated using [`phasespace`](https://github.com/zfit/phasespace) python package.

Phase space represents the set of all possible states (positions and momenta) of a system.
It is critical for understanding the outcomes of particle collisions, decays, and interactions.
It allows physicists to calculate the likelihood of various final states given a set of initial conditions, taking into account conservation laws (energy, momentum, angular momentum, etc.).
The volume of phase space available for a particular process is directly related to the probability of that process occurring.

Firstly, in CM frame the 4-momentum of the total system can be acquired by 4-momentum conservation:

$$\begin{pmatrix}
 E_0 \\
 0 \\
 0 \\
 0
\end{pmatrix}
= \begin{pmatrix}
 E_{\gamma} \\
 0 \\
 0 \\
 p_z
\end{pmatrix} +
\begin{pmatrix}
 E_p \\
 0 \\
 0 \\
 -p_z
\end{pmatrix}
= \begin{pmatrix}
 E_{\eta} \\
 p_{\eta,x} \\
 p_{\eta,y} \\
 p_{\eta,z}
\end{pmatrix} +
\begin{pmatrix}
 E_{\pi} \\
 p_{\pi,x} \\
 p_{\pi,y} \\
 p_{\pi,z}
\end{pmatrix} +
\begin{pmatrix}
 E_p \\
 p_{p,x} \\
 p_{p,y} \\
 p_{p,z}
\end{pmatrix}
$$ (4momentum_label)

For the record, we need to add parameters of beam momentum and CM total energy

:::{caution}
The calculation here is in the CM frame, and usually for experiment it is in lab frame.
:::

$m_{\gamma}=0$, 

$E_{\gamma} = p_{\gamma}$

$E_{p} = \sqrt { p_{p}^2+ m_{p}^2}$

$m_0 = E_0 = |p_z|+\sqrt{p_z^2+m_p^2}$

The $\gamma$ energy ($E_{\gamma}$) and $\gamma$ beam momentum ($p_{\gamma}$) are as input in this analysis, they are:

$E_{\gamma} = p_{\gamma} = 1.943718 GeV$

:::{important}
Beam momentum (energy) as input value. Assuming incoming $\gamma$ beam to stationary proton target in photo-production experiment, e.g. GlueX.
:::

In [ ]:
p_beam = 1.943718
m_proton = 0.938
m_0 = p_beam + np.sqrt(p_beam**2 + m_proton**2)
m_eta = 0.548
m_pi = 0.135

In [ ]:
m_0

Thus, we then have the mass of the system $m_0$ (or $m_{p\gamma}$ in this case) in CM frame:

$$
E_{0} = m_0\approx 4.102 \;\; GeV 
$$

Use of `phasespace` to generate decay particles (4-momentum and weights):

In [ ]:
import phasespace

pγ_mass = m_0
eta_mass = m_eta
pi_mass = m_pi
p_mass = m_proton

weights, particles = phasespace.nbody_decay(
    pγ_mass, [eta_mass, pi_mass, p_mass]
).generate(n_events=phsp_events)

### 4 momentum of decay particles

The 4-momentum of decay particles $\eta$ (1), $\pi$ (2), and $p$ (3) in phase space to be generated:

In [ ]:
import tensorflow as tf

n_final_state = 3


def generate_phsp_decay(
    size: int, seed: int | None = None, bunch_size: int | None = None
) -> tuple[MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D]:
    if bunch_size is None:
        bunch_size = size
    rng = np.random.default_rng(seed=seed)
    generated_seed = rng.integers(1_000_000)
    phsp_sample = generate_phsp_bunch(size, generated_seed)
    while get_size(phsp_sample) < size:
        bunch = generate_phsp_bunch(bunch_size, generated_seed)
        phsp_sample = concatenate(phsp_sample, bunch)
    phsp_sample = remove_overflow(phsp_sample, size)
    return tuple(to_vector(tensor) for tensor in phsp_sample)


def generate_phsp_bunch(
    size: int, seed: int | None = None
) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    rng = np.random.default_rng(seed=seed)
    weights, particles = phasespace.nbody_decay(
        pγ_mass, [eta_mass, pi_mass, p_mass]
    ).generate(n_events=size, seed=seed)
    random_weights = rng.uniform(0, weights.numpy().max(), size=weights.shape)
    selector = weights > random_weights
    return tuple(particles[f"p_{i}"][selector] for i in range(n_final_state))


def get_size(phsp: tuple[tf.Tensor, tf.Tensor, tf.Tensor]):
    return len(phsp[0])


def concatenate(
    phsp1: tuple[tf.Tensor, tf.Tensor, tf.Tensor],
    phsp2: tuple[tf.Tensor, tf.Tensor, tf.Tensor],
) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    return tuple(tf.concat([phsp1[i], phsp2[i]], axis=0) for i in range(3))


def remove_overflow(phsp: tuple[tf.Tensor, tf.Tensor, tf.Tensor], size: int):
    return tuple(tensor[:size] for tensor in phsp)


def to_vector(tensor: tf.Tensor) -> MomentumNumpy4D:
    return vector.array({
        key: tensor.numpy().T[j] for j, key in enumerate(["px", "py", "pz", "E"])
    })

A few functions are defined above for generating phase space samples of particle decays and converting them into four-momentum vectors. The main ideas of them are:

- Step 1, Generate Phase Space Sample (`generate_phsp_bunch()`)
    - Random Number Generator
    - Phase Space Generation
    - Rejection Sampling
      
- Step 2, Ensure Sufficient Sample Size (`generate_phsp_decay()`, `get_size()`, `concatenate()`, `remove_overflow()` )
    - Iterative Sampling
    - Removing Excess

- Step 3, Convert to Four-Momentum Vectors (`to_vector()`)
    - Four-Momentum Representation

In [ ]:
%%time
p1_phsp_test, p2_phsp_test, p3_phsp_test = generate_phsp_decay(
    size=phsp_events, seed=42
)

### 4 momentum of reaction particles

The 4 momentum of reaction particles $\gamma$ (a) and $p$ (b) are generated.

To find $p_a$ and $p_b$ by 4-momentum conservation, energy conservation in this case.
With:

$m_a = m_{\gamma} =0$,

$m_b = m_p$,

$p_{a,x} = p_{a,y} = 0 = p_{b,x} = p_{b,y}$

and

$p_{a,z} = - p_{b,z}$


$$
E_a + E_b = E_1 + E_2 + E_3 = E_0
$$

$$
\sqrt{m_a^2 + p_a^2} + \sqrt{m_b^2 + p_b^2} = E_0
$$

$$
p_a + \sqrt{m_b^2 + (-p_a)^2} = E_0
$$

$$
p_{a,z} + \sqrt{m_p^2 + p_a^2} - E_0 = 0
$$

thus

$$
p_{a,z} = \frac{E_0^2 - m_p^2}{2E_0}
$$

then

$$
p_{b,z} = -p_{a,z}
$$

$$
E_a =p_a
$$

$$
E_{b} = \sqrt{m_b^2+p_b^2}
$$



In [ ]:
def compute_pa_pb(
    p1_phsp: MomentumNumpy4D, p2_phsp: MomentumNumpy4D, p3_phsp: MomentumNumpy4D
) -> tuple[MomentumNumpy4D, MomentumNumpy4D]:
    shape = p1_phsp.shape
    E_0 = (p1_phsp + p2_phsp + p3_phsp).e
    px = np.zeros(shape)
    py = np.zeros(shape)
    pz = np.ones(shape) * (E_0**2 - p3_phsp.m.mean() ** 2) / (2 * E_0)
    E = np.ones(shape) * np.sqrt((p3_phsp.m.mean()) ** 2 + pz.mean() ** 2)
    pa_phsp = MomentumNumpy4D({"E": pz, "px": px, "py": py, "pz": pz})
    pb_phsp = MomentumNumpy4D({"E": E, "px": px, "py": py, "pz": -pz})
    return [pa_phsp, pb_phsp]

In [ ]:
%%time
pa_phsp_test, pb_phsp_test = compute_pa_pb(p1_phsp_test, p2_phsp_test, p3_phsp_test)

### 4 momentum of all particles

In [ ]:
def generate_phsp_all(
    size: int, seed: int | None = None, bunch_size: int | None = None
) -> tuple[
    MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D
]:
    p1_phsp, p2_phsp, p3_phsp = generate_phsp_decay(size, seed, bunch_size)
    pa_phsp, pb_phsp = compute_pa_pb(p1_phsp, p2_phsp, p3_phsp)
    return [p1_phsp, p2_phsp, p3_phsp, pa_phsp, pb_phsp]

In [ ]:
%%time
p1_phsp, p2_phsp, p3_phsp, pa_phsp, pb_phsp = generate_phsp_all(
    size=phsp_events, seed=42
)

In [ ]:
p0_phsp = pa_phsp + pb_phsp
p12_phsp = p1_phsp + p2_phsp
p23_phsp = p2_phsp + p3_phsp
p31_phsp = p3_phsp + p1_phsp

s12_phsp = p12_phsp.m2
s23_phsp = p23_phsp.m2
s31_phsp = p31_phsp.m2
t1_phsp = (pa_phsp - p1_phsp).m2
t2_phsp = (pa_phsp - p2_phsp).m2
u3_phsp = (pb_phsp - p3_phsp).m2

Calculations with 4-vectors are performed with the [`vector`](https://vector.readthedocs.io/en/latest/usage/intro.html) package:

In [ ]:
np.testing.assert_almost_equal(
    s12_phsp,
    (p1_phsp.e + p2_phsp.e) ** 2
    - (
        p1_phsp.p**2
        + p2_phsp.p**2
        + 2
        * (p1_phsp.px * p2_phsp.px + p1_phsp.py * p2_phsp.py + p1_phsp.pz * p2_phsp.pz)
    ),
)

In [ ]:
np.testing.assert_almost_equal(p0_phsp.p2.max(), 0)

In [ ]:
np.testing.assert_almost_equal(
    t1_phsp,
    (pa_phsp.e - p1_phsp.e) ** 2
    - (
        pa_phsp.p**2
        + p1_phsp.p**2
        - 2
        * (pa_phsp.px * p1_phsp.px + pa_phsp.py * p1_phsp.py + pa_phsp.pz * p1_phsp.pz)
    ),
)
np.testing.assert_almost_equal(
    t2_phsp,
    (pa_phsp.e - p2_phsp.e) ** 2
    - (
        pa_phsp.p**2
        + p2_phsp.p**2
        - 2
        * (pa_phsp.px * p2_phsp.px + pa_phsp.py * p2_phsp.py + pa_phsp.pz * p2_phsp.pz)
    ),
)
np.testing.assert_almost_equal(
    u3_phsp,
    (pb_phsp.e - p3_phsp.e) ** 2
    - (
        pb_phsp.p**2
        + p3_phsp.p**2
        - 2
        * (pb_phsp.px * p3_phsp.px + pb_phsp.py * p3_phsp.py + pb_phsp.pz * p3_phsp.pz)
    ),
)

### Dalitz Plot of Phase Space

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.suptitle("Phase Space Dalitz Plot")
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05])  # The last column for colorbar
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
cax = plt.subplot(gs[2])  # For colorbar

hist2 = ax2.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    cmin=1e-2,
    density=True,
    cmap="jet",
    range=[[0, 10.3], [0.5, 13]],
)
ax2.set_title("2D Histogram")
ax2.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")

scat1 = ax1.scatter(s12_phsp, s23_phsp, s=1e-4, c="black", norm=mpl.colors.Normalize())
ax1.set_xlabel(R"$m^2(\eta \pi^0)\;\left[\mathrm{GeV}^2\right]$")
ax1.set_ylabel(R"$m^2(\pi^0 p)\;\left[\mathrm{GeV}^2\right]$")
ax1.set_title("Scatter Plot")

cbar2 = fig.colorbar(hist2[3], cax=cax)
fig.tight_layout()
fig.show()

:::{note}
Different choice of plotting Dalitz plot: 2D histogram or scatter plot.
:::

## Calculation of Angles

### CM Angles 

Angles in CM frame (CM Angles) are the polar angles and azimuthal right after the phase space generation (and also after data generation) in CM frame, they are different than the helicity angles in the subsystem (which is after rotation and boost into the subsystem).

The vales for phase space can be obtained directly after the generation 

$(\theta_1^{\text{CM}}, \theta_2^{\text{CM}}, \theta_3^{\text{CM}}, \phi_1^{\text{CM}}, \phi_2^{\text{CM}}, \phi_3^{\text{CM}})$:

In [ ]:
theta1_CM_phsp = p1_phsp.theta
theta2_CM_phsp = p2_phsp.theta
theta3_CM_phsp = p3_phsp.theta
phi1_CM_phsp = p1_phsp.phi
phi2_CM_phsp = p2_phsp.phi
phi3_CM_phsp = p3_phsp.phi

### Helicity angles

The Helicity angle between decay products(Polar Angles), by definition:

$$
\theta = \arccos \frac{p_z}{|p|}
$$

The Helicity angle with the production plane and decay plane(Azimuthal Angles), by definition:

$$
\phi = \arctan2(p_y , p_x)
$$

:::{important}
Theses angles are in helicity formalism, the calculation involved rotation and boost into the subsystem as following:
:::

Before boosting (in CM frame)

<iframe src="https://www.geogebra.org/3d/dgjn83pb?embed" width="800" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

After Boosted into system$_{12}$ rest frame (it is similar for boosting into system$_{23}$ and system$_{31}$)

<iframe src="https://www.geogebra.org/3d/tv5kr8pp?embed" width="800" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

The helicity angle between decay product before boost:

The helicity angles $\Omega_i$ are pairs of Euler angles $\Omega_i = \left(\theta_i, \phi_i\right)$:
- Opening angle $\theta_1 \equiv \theta^{12}_1$ of four-momentum $p_1$ is the angle between $p'_1 \equiv p^{12}_1$ and $p_{12}$.
- The angle $\phi_1 \equiv \phi^{12}_1$ defines the angle between the **production plane** spanned by $p_{12}$ and $p_3$ and the **decay plane** spanned by $p^{(\prime)}_1$ and $p^{(\prime)}_2$.

The helicity angles of the other subsystems are defined by cyclic permutation of the indices, such that $\theta_2 \equiv \theta^{12}_2$ and $\theta_3 \equiv \theta^{31}_1$.

To calculate the helicity angles $\theta$ and $\phi$, a combination of boost and rotation (in Y and Z axis) in CM frame is defined:

In [ ]:
def theta_helicity(p_i: MomentumNumpy4D, p_ij: MomentumNumpy4D):
    return p_i.rotateZ(-p_ij.phi).rotateY(-p_ij.theta).boostZ(-p_ij.beta).theta


def phi_helicity(p_i, p_ij):
    return p_i.rotateZ(-p_ij.phi).rotateY(-p_ij.theta).boostZ(-p_ij.beta).phi

In [ ]:
theta1_phsp = theta_helicity(p1_phsp, p12_phsp)
theta2_phsp = theta_helicity(p2_phsp, p23_phsp)
theta3_phsp = theta_helicity(p3_phsp, p31_phsp)
phi1_phsp = phi_helicity(p1_phsp, p12_phsp)
phi2_phsp = phi_helicity(p2_phsp, p23_phsp)
phi3_phsp = phi_helicity(p3_phsp, p31_phsp)

## Distributions in phase space

The distribution of angular projection and invariant mass in phase space is shown in this [section](#my_section) together with data and weight phase space (model) after data generation

In [ ]:
def plot_helicity_angles_2d(
    phi1, phi2, phi3, theta1, theta2, theta3, title: str
) -> None:
    fig, axes = plt.subplots(figsize=(13, 4), ncols=3, sharey=True)
    for i, ax in enumerate(axes, 1):
        ax.set_xlabel(Rf"$\theta_{i}$")
        ax.set_ylabel(Rf"$\phi_{i}$")
    axes[0].hist2d(theta1[~np.isnan(phi1)], phi1[~np.isnan(phi1)], bins=100)
    axes[1].hist2d(theta2[~np.isnan(phi2)], phi2[~np.isnan(phi2)], bins=100)
    axes[2].hist2d(theta3[~np.isnan(phi3)], phi3[~np.isnan(phi3)], bins=100)
    fig.suptitle(title)
    fig.tight_layout()
    plt.show()


plot_helicity_angles_2d(
    phi1_phsp,
    phi2_phsp,
    phi3_phsp,
    theta1_phsp,
    theta2_phsp,
    theta3_phsp,
    title="Phase space",
)

The CM angles, helicity angles (in subsystem), and invariant mass in phase space are investigated:

In [ ]:
fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)

for i, ax in enumerate(thetaCM_ax, 1):
    ax.set_title(Rf"$\cos (\theta_{i}^{{CM}})$")
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phiCM_ax, 1):
    ax.set_title(Rf"$\phi_{i}^{{CM}}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])

for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\cos (\theta_{i}^{{{i}{(i % 3 + 1)}}})$")
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}^{{{i}{(i % 3 + 1)}}}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")


thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    label="phsp",
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    label="phsp",
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    label="phsp",
)

phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=50,
    label="phsp",
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=50,
    label="phsp",
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=50,
    label="phsp",
)

theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    label="phsp",
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    label="phsp",
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    label="phsp",
)

phi_ax[0].hist(
    phi1_phsp,
    bins=50,
    label="phsp",
)
phi_ax[1].hist(
    phi2_phsp,
    bins=50,
    label="phsp",
)
phi_ax[2].hist(
    phi3_phsp,
    bins=50,
    label="phsp",
)

mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    label="phsp",
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    label="phsp",
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    label="phsp",
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.tight_layout()
plt.show()

## Spherical harmonics for the model

The calculation of $Y_l^m(\phi, \theta)$ is done via `scipy.special.sph_harm(m, l, phi, theta)` 

We have changed the use of symbol $\phi$ and $\theta$ according to the more common used definition in physics.

here the notation of $\theta$ and $\phi$ are not using the same as in `scipy` special function [scipy.special.sph_harm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html)

(in `scipy` $\theta$ is the azimuthal  from -$\pi$ to $\pi$ and $\phi$ is the polar angle from 0 to $\pi$)

where here (more common in physics):

$\phi$ is the azimuthal  from -$\pi$ to $\pi$ (in `scipy` it is $\theta$ and from 0 to $2\pi$)

$\theta$ is the polar angle from 0 to $\pi$ (in `scipy` it is $\phi$)

$Y_l^m(\phi, \theta) = \sqrt{\frac{2n+1}{4\pi}\frac{(n-m)!}{(n+m)!}}e^{im\phi}P_l^m(\cos(\theta))$

In [ ]:
initial_SH_parameters = dict(
    a_minus2=0,
    a_minus1=0.5,
    a_0=3.5,
    a_plus1=4,
    a_plus2=2.5,
    b_minus1=-1.5,
    b_0=4,
    b_plus1=0.5,
    c_0=2.5,
)

In [ ]:
a_minus2 = initial_SH_parameters.get("a_minus2")
a_minus1 = initial_SH_parameters.get("a_minus1")
a_0 = initial_SH_parameters.get("a_0")
a_plus1 = initial_SH_parameters.get("a_plus1")
a_plus2 = initial_SH_parameters.get("a_plus2")

In [ ]:
def compute_spherical_harmonics12(
    theta: np.ndarray,
    phi: np.ndarray,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    **kwargs,
) -> np.ndarray:
    return (
        a_plus2 * sp.special.sph_harm(2, 2, phi, theta)
        + a_plus1 * sp.special.sph_harm(1, 2, phi, theta)
        + a_0 * sp.special.sph_harm(0, 2, phi, theta)
        + a_minus1 * sp.special.sph_harm(-1, 2, phi, theta)
        + a_minus2 * sp.special.sph_harm(-2, 2, phi, theta)
    )

In [ ]:
PHI, THETA = np.meshgrid(
    np.linspace(-np.pi, +np.pi, num=1_000),
    np.linspace(0, np.pi, num=1_000),
)
Z12 = compute_spherical_harmonics12(
    PHI,
    THETA,
    **initial_SH_parameters,
    # PHI, THETA, a_minus2, a_minus1, a_0, a_plus1, a_plus2
)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z12.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z12.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Real Part of $\sum a_m Y_2^m (\Omega_1)$")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Imaginary Part of $\sum a_m Y_2^m (\Omega_1)$")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
plt.hist2d(
    p12_phsp.phi,
    p12_phsp.theta,
    bins=100,
    weights=compute_spherical_harmonics12(
        p12_phsp.phi, p12_phsp.theta, a_minus2, a_minus1, a_0, a_plus1, a_plus2
    ).real,
    cmap=plt.cm.coolwarm,
)
plt.title("$p_{12}$ with real part of spherical harmonics as weights in histogram")
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\theta$")
plt.show()

In [ ]:
b_minus1 = initial_SH_parameters.get("b_minus1")
b_0 = initial_SH_parameters.get("b_0")
b_plus1 = initial_SH_parameters.get("b_plus1")

In [ ]:
def compute_spherical_harmonics23(
    theta: np.ndarray, phi: np.ndarray, b_minus1, b_0, b_plus1, **kwargs
) -> np.ndarray:
    return (
        b_plus1 * sp.special.sph_harm(1, 1, phi, theta)
        + b_0 * sp.special.sph_harm(0, 1, phi, theta)
        + b_minus1 * sp.special.sph_harm(-1, 1, phi, theta)
    )

In [ ]:
Z23 = compute_spherical_harmonics23(PHI, THETA, **initial_SH_parameters)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z23.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z23.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Real Part of $\sum b_m Y_1^m (\Omega_2)$")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Imaginary Part of $\sum b_m Y_1^m (\Omega_2)$")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
c_0 = initial_SH_parameters.get("c_0")

In [ ]:
def compute_spherical_harmonics31(
    theta: np.ndarray, phi: np.ndarray, c_0, **kwargs
) -> np.ndarray:
    return c_0 * sp.special.sph_harm(0, 0, phi, theta)

:::{note}
The Wigner d-function can be related to the associated Legendre polynomials
:::

$Y_l^m(\theta,\phi) = \sqrt{\frac{2l+1}{4\pi}\frac{(l-m)!}{(l+m)!}}e^{im\phi} d^l_{m0}(\theta)$

In [ ]:
from scipy.special import lpmv


def wigner_d_function(j, m, n, beta):
    """
    Calculate the Wigner d-function for given j, m, n, and beta.

    Parameters:
    j (int): Total angular momentum quantum number.
    m, n (int): Magnetic quantum numbers.
    beta (float): The angle (in radians).

    Returns:
    float: The value of the Wigner d-function.
    """
    # The Wigner d-function can be related to the associated Legendre polynomials
    # Here, we use scipy's lpmv function which computes the associated Legendre polynomials
    return (
        np.sqrt(
            (np.math.factorial(j + m) * np.math.factorial(j - m))
            / (np.math.factorial(j + n) * np.math.factorial(j - n))
        )
        * np.cos(beta / 2) ** (2 * j + n - m)
        * np.sin(beta / 2) ** (m - n)
        * lpmv(abs(n - m), 2 * j, np.cos(beta))
    )


def spherical_harmonics(l_num, m, theta, phi):
    """
    Calculate the spherical harmonics using the Wigner d-function.

    Parameters:
    l (int): Angular momentum quantum number.
    m (int): Magnetic quantum number.
    theta (float): Polar angle in radians.
    phi (float): Azimuthal angle in radians.

    Returns:
    complex: The value of the spherical harmonic.
    """
    # Calculating the spherical harmonics using Wigner d-function
    # Y^m_l(θ, φ) = √((2l+1)/(4π)) * e^(-imφ) * d^l_{m0}(θ)
    normalization = np.sqrt((2 * l_num + 1) / (4 * np.pi))
    return normalization * np.exp(1j * m * phi) * wigner_d_function(l_num, m, 0, theta)

In [ ]:
sp.special.sph_harm(1, 1, phi1_phsp, theta1_phsp)

In [ ]:
spherical_harmonics(1, 1, theta1_phsp, phi1_phsp)

## Implementation of the models

### Parameter Default initial values

The original values of initial guess can be obtained from the data file from the [Lecture 11 in STRONG2020 HaSP School](https://indico.ific.uv.es/event/6803/contributions/21223/).
But the values are modified here to make the strides in Dalitz plot more visible.

The purpose is to define and test the model from fewer to more terms.

The initial default parameters are resonance mass $(M)$ and width $(\Gamma)$:

$M_{12}$ = $M_{\eta\pi^0}$ = $M_{a_2}$

$M_{23}$ = $M_{\pi^0p}$ = $M_{\Delta^+}$

$M_{31}$ = $M_{\eta p}$ = $M_{N^*}$

$\Gamma_{12}$ = $\Gamma_{\eta\pi^0}$ = $\Gamma_{a_2}$

$\Gamma_{23}$ = $\Gamma_{\pi^0p}$ = $\Gamma_{\Delta^+}$

$\Gamma_{31}$ = $\Gamma_{\eta p}$ = $\Gamma_{N^*}$

In [ ]:
initial_parameters = dict(
    M12=1.32,
    Gamma12=0.1,
    M23=1.24 + 0.3,
    Gamma23=0.1,
    M31=1.57 + 0.3,
    Gamma31=0.1,
    a_minus2=initial_SH_parameters.get("a_minus2"),
    a_minus1=initial_SH_parameters.get("a_minus1"),
    a_0=initial_SH_parameters.get("a_0"),
    a_plus1=initial_SH_parameters.get("a_plus1"),
    a_plus2=initial_SH_parameters.get("a_plus2"),
    b_minus1=initial_SH_parameters.get("b_minus1"),
    b_0=initial_SH_parameters.get("b_0"),
    b_plus1=initial_SH_parameters.get("b_plus1"),
    c_0=initial_SH_parameters.get("c_0"),
)

M12 = initial_parameters.get("M12")
Gamma12 = initial_parameters.get("Gamma12")
M23 = initial_parameters.get("M23")
Gamma23 = initial_parameters.get("Gamma23")
M31 = initial_parameters.get("M31")
Gamma31 = initial_parameters.get("Gamma31")

### Breit-Wigner (only) Model

In [ ]:
def BW_model(s12, s23, s31, *, M12, Gamma12, M23, Gamma23, M31, Gamma31, **kwargs):
    A12 = BW(s12, M12, Gamma12)
    A23 = BW(s23, M23, Gamma23)
    A31 = BW(s31, M31, Gamma31)
    return np.abs(A12 + A23 + A31) ** 2


def BW(s, m, Gamma):
    return 1 / (s - m**2 + m * Gamma * 1j)

### Spherical Harmonics (only) Model

In [ ]:
def SH_model(
    phi1,
    theta1,
    phi2,
    theta2,
    *,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
    **kwargs,
):
    return (
        np.abs(
            compute_spherical_harmonics12(
                phi1, theta1, a_minus2, a_minus1, a_0, a_plus1, a_plus2
            )
            + compute_spherical_harmonics23(phi2, theta2, b_minus1, b_0, b_plus1)
            + c_0
        )
        ** 2
    )

### Breit-Wigner $\times$ Spherical Harmonics Model

In [ ]:
def BW_SH_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    *,
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
    **kwargs,
):
    A12 = BW(s12, M12, Gamma12) * compute_spherical_harmonics12(
        phi1, theta1, a_minus2, a_minus1, a_0, a_plus1, a_plus2
    )
    A23 = BW(s23, M23, Gamma23) * compute_spherical_harmonics23(
        phi2, theta2, b_minus1, b_0, b_plus1
    )
    A31 = BW(s31, M31, Gamma31) * c_0
    return np.abs(A12 + A23 + A31) ** 2

### Full Model

In [ ]:
def full_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    t1,
    t2,
    u3,
    *,
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
    **kwargs,
):
    A12 = (
        BW(s12, M12, Gamma12)
        * compute_spherical_harmonics12(
            phi1, theta1, a_minus2, a_minus1, a_0, a_plus1, a_plus2
        )
        * s12 ** (0.5 + 0.9 * u3)
    )
    A23 = (
        BW(s23, M23, Gamma23)
        * compute_spherical_harmonics23(phi2, theta2, b_minus1, b_0, b_plus1)
        * s23 ** (0.5 + 0.9 * t1)
    )
    A31 = BW(s31, M31, Gamma31) * c_0 * s31 ** (1.08 + 0.2 * t2)
    return np.abs(A12 + A23 + A31) ** 2

### Dalitz Plots of (sub)models

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(R"Dalitz Plots of sub-models")

hist1 = ax1.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=BW_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
)
ax1.set_xlabel(R"$m^2(\eta \pi^0)$")
ax1.set_title("BW model only")
ax1.set_ylabel(R"$m^2(\pi^0 p)$")

hist2 = ax2.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=SH_model(
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
)
ax2.set_title("SH model only")
ax2.set_xlabel(R"$m^2(\eta \pi^0)$")
ax2.set_ylabel(R"$m^2(\pi^0 p)$")


hist3 = ax3.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=BW_SH_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
)
ax3.set_title(R"BW $\times$ SH model")
ax3.set_xlabel(R"$m^2(\eta \pi^0)$")
ax3.set_ylabel(R"$m^2(\pi^0 p)$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
fig.suptitle(R"Dalitz Plot of BW $\times$ SH model with cut at max=0.15")
hist = ax.hist2d(
    s12_phsp,
    s23_phsp,
    bins=200,
    weights=BW_SH_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
)

ax.set_xlabel(r"$m^2(\eta \pi^0)$")
ax.set_ylabel(r"$m^2(\pi^0 p)$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1) = plt.subplots(figsize=(3.5, 3), ncols=1, sharey=True)
hist1 = ax1.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=full_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        t1_phsp,
        t2_phsp,
        u3_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-8,
    density=True,
    cmap="jet",
)
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
ax1.set_title("Full model")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
fig.tight_layout()
fig.show()

In [ ]:
weight_BW = BW_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    M12=M12,
    Gamma12=Gamma12,
    M23=M23,
    Gamma23=Gamma23,
    M31=M31,
    Gamma31=Gamma31,
)
weight_SH = SH_model(
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    a_0=a_0,
    a_minus2=a_minus2,
    a_minus1=a_minus1,
    a_plus1=a_plus1,
    a_plus2=a_plus2,
    b_minus1=b_minus1,
    b_0=b_0,
    b_plus1=b_plus1,
    c_0=c_0,
)

weight_BW_SH = BW_SH_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    M12=M12,
    Gamma12=Gamma12,
    M23=M23,
    Gamma23=Gamma23,
    M31=M31,
    Gamma31=Gamma31,
    a_minus2=a_minus2,
    a_minus1=a_minus1,
    a_0=a_0,
    a_plus1=a_plus1,
    a_plus2=a_plus2,
    b_minus1=b_minus1,
    b_0=b_0,
    b_plus1=b_plus1,
    c_0=c_0,
)
weight_Full = full_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    t1_phsp,
    t2_phsp,
    u3_phsp,
    M12=M12,
    Gamma12=Gamma12,
    M23=M23,
    Gamma23=Gamma23,
    M31=M31,
    Gamma31=Gamma31,
    a_minus2=a_minus2,
    a_minus1=a_minus1,
    a_0=a_0,
    a_plus1=a_plus1,
    a_plus2=a_plus2,
    b_minus1=b_minus1,
    b_0=b_0,
    b_plus1=b_plus1,
    c_0=c_0,
)

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1_phsp,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
theta_ax[1].hist(
    theta2_phsp,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
theta_ax[2].hist(
    theta3_phsp,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
theta_ax[0].hist(
    theta1_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[1].hist(
    theta2_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[2].hist(
    theta3_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[0].hist(
    theta1_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[1].hist(
    theta2_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[2].hist(
    theta3_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[0].hist(
    theta1_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    theta2_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    theta3_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    linestyle="dashed",
    label="phsp",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass: phsp and models with good initial guess of"
    r" parameters"
)
fig.tight_layout()
plt.show()

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1_phsp,
    bins=100,
    weights=weight_Full,
    label="full",
)
theta_ax[1].hist(
    theta2_phsp,
    bins=100,
    weights=weight_Full,
    label="full",
)
theta_ax[2].hist(
    theta3_phsp,
    bins=100,
    weights=weight_Full,
    label="full",
)

phi_ax[0].hist(
    phi1_phsp,
    bins=50,
    weights=weight_Full,
    label="full",
)
phi_ax[1].hist(
    phi2_phsp,
    bins=50,
    weights=weight_Full,
    label="full",
)
phi_ax[2].hist(
    phi3_phsp,
    bins=50,
    weights=weight_Full,
    label="full",
)

mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_Full,
    label="full",
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_Full,
    label="full",
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_Full,
    label="full",
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass (with the same good initial guess of"
    r" parameters previously )"
)
fig.tight_layout()
plt.show()

## Data Generation

### Hit and miss intensity sample

This function generate a data sample based on a model by using hit and miss filter applied to the phase space sample. 

The output is 4-vector in the form of a `tuple`[$p_1$,$p_2$,$p_3$,$p_a$,$p_b$].

In [ ]:
def generate_data(
    model: callable, size: int, bunch_size: int | None = None, seed: int | None = None
) -> tuple[MomentumNumpy4D, ...]:
    if bunch_size is None:
        bunch_size = size
    rng = np.random.default_rng(seed=seed)
    generated_seed = rng.integers(1_000_000)
    phase_space = generate_phsp_all(size, generated_seed)
    data_sample = _hit_and_miss_filter(model, phase_space, generated_seed)
    while get_size_data(data_sample) < size:
        phase_space = generate_phsp_all(bunch_size, generated_seed)
        bunch = _hit_and_miss_filter(model, phase_space)
        data_sample = concatenate_data(data_sample, bunch)
    return remove_overflow_data(data_sample, size)


def _hit_and_miss_filter(
    model: callable, phase_space: tuple[MomentumNumpy4D, ...], seed: int | None = None
) -> tuple[MomentumNumpy4D, ...]:
    p1, p2, p3, pa, pb = phase_space
    p12 = p1 + p2
    p23 = p2 + p3
    p31 = p3 + p1

    intensities: np.ndarray = model(
        s12=p12.m2,
        s23=p23.m2,
        s31=p31.m2,
        t1=(pa - p1).m2,
        t2=(pa - p2).m2,
        u3=(pb - p3).m2,
        phi1=phi_helicity(p1, p12),
        theta1=theta_helicity(p1, p12),
        phi2=phi_helicity(p2, p23),
        theta2=theta_helicity(p2, p23),
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    )
    rng = np.random.default_rng(
        seed=seed
    )  # Fixed seed is used here for Reproducibility and Testing
    random_intensities = rng.uniform(0, intensities.max(), size=intensities.shape)
    selector = intensities > random_intensities
    return (
        p1[selector],
        p2[selector],
        p3[selector],
        pa[selector],
        pb[selector],
    )


def get_size_data(
    data: tuple[
        MomentumNumpy4D,
        MomentumNumpy4D,
        MomentumNumpy4D,
        MomentumNumpy4D,
        MomentumNumpy4D,
    ],
):
    return len(data[0])


def concatenate_data(
    data1: tuple[MomentumNumpy4D, ...],
    data2: tuple[MomentumNumpy4D, ...],
) -> tuple[MomentumNumpy4D, ...]:
    return tuple(concatenate_vectors((pi1, pj2)) for pi1, pj2 in zip(data1, data2))


def concatenate_vectors(vectors: tuple[MomentumNumpy4D]) -> MomentumNumpy4D:
    return vector.array({
        "px": np.concatenate([p.px for p in vectors]),
        "py": np.concatenate([p.py for p in vectors]),
        "pz": np.concatenate([p.pz for p in vectors]),
        "E": np.concatenate([p.e for p in vectors]),
    })


def remove_overflow_data(
    data: tuple[MomentumNumpy4D, ...], size: int
) -> tuple[MomentumNumpy4D, ...]:
    return tuple(momentum[:size] for momentum in data)

A few functions are defined above for generating data sample from filtering phase space samples of particle decays and converting them into four-momentum vectors. The main ideas of them are:

- `generate_data()`: The main function used to generate the data 
    - It simulates events until it gathers a specified number of valid data points that meet the model’s conditions.
    - Parameters:
        - model: A callable that computes the intensity of events based on given kinematic variables.
        - size: The desired number of valid events to generate.
        - bunch_size: The number of events to generate per iteration if the initial batch doesn't meet the required size. Defaults to size if not provided.
        - seed: Seed for random number generation to ensure reproducibility.
    - Process:
        - Initialize a random number generator.
        - Generate an initial set of phase space data (phase_space) and filter it using the model to obtain valid events (data_sample).
        - If the filtered data (data_sample) is less than the desired size, additional data is generated in batches (bunch_size) and filtered until the size requirement is met.
        - The function ensures the returned dataset size matches exactly the requested size.
     

- `_hit_and_miss_filter`: The other important function that filters the generated phase space data
    - using the hit-and-miss Monte Carlo method based on the model intensities.
    - Parameters:
        - model: The model function to calculate intensities.
        - phase_space: Tuple containing momentum vectors of particles involved in the event.
        - seed: Seed for random number generation.
    - Process:
        - Calculate invariant masses and other kinematic variables required by the model.
        - Call the model with these variables to compute intensities for each event.
        - Generate random numbers and select events where the model intensity is greater than a random threshold, simulating a probability proportional to the intensity.
     
There are other assisted functions that are used in the functions above:

    - `get_size_data`:
        - Simply returns the number of events in the data, used to check if more data needs to be generated.

    - `concatenate_data`:
        - Merges two datasets into one, maintaining the structure of tuples of momentum vectors.

    - `concatenate_vectors`:
        - Helper function to concatenate individual momentum vectors within the data tuples.

    - `remove_overflow_data`:
        - Trims the dataset to the desired size if it exceeds the specified number due to batch generation.

In [ ]:
%%time
data = generate_data(BW_SH_model, data_events, seed=0)

In [ ]:
p1_data, p2_data, p3_data, pa_data, pb_data = data

In [ ]:
p12_data = p1_data + p2_data
p23_data = p2_data + p3_data
p31_data = p3_data + p1_data

s12_data = p12_data.m2
s23_data = p23_data.m2
s31_data = p31_data.m2
t1_data = (pa_data - p1_data).m2
t2_data = (pa_data - p2_data).m2
u3_data = (pb_data - p3_data).m2

theta1_CM_data = p1_data.theta
theta2_CM_data = p2_data.theta
theta3_CM_data = p3_data.theta
phi1_CM_data = p1_data.phi
phi2_CM_data = p2_data.phi
phi3_CM_data = p3_data.phi

theta1_data = theta_helicity(p1_data, p12_data)
theta2_data = theta_helicity(p2_data, p23_data)
theta3_data = theta_helicity(p3_data, p31_data)
phi1_data = phi_helicity(p1_data, p12_data)
phi2_data = phi_helicity(p2_data, p23_data)
phi3_data = phi_helicity(p3_data, p31_data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
fig.suptitle("Dalitz Plot of Generated Data")
hist = ax.hist2d(s12_data, s23_data, bins=100, cmin=1e-6, density=True, cmap="jet")
ax.set_xlabel(r"$m^2_{\eta \pi^0}$")
ax.set_ylabel(r"$m^2_{\pi^0 p}$")
cbar = fig.colorbar(hist[3], ax=ax)
fig.tight_layout()
fig.show()

(my_section)=
## Distribution in weighted phase space (model), phase space, and data 

The distribution of CM angles, the helicity angles and invariant mass are shown in this section. 

The angles in phase space, data, and models with good initial guess parameters are all obtained for comparison.

In [ ]:
thetaCM_subtitles = [
    R"$\cos (\theta_{1}^{{CM}}) \equiv \cos (\theta_{\eta}^{{CM}})$",
    R"$\cos (\theta_{2}^{{CM}}) \equiv \cos (\theta_{\pi^0}^{{CM}})$",
    R"$\cos (\theta_{3}^{{CM}}) \equiv \cos (\theta_{p}^{{CM}})$",
]
phiCM_subtitles = [
    R"$\phi_1^{CM} \equiv \phi_{\eta}^{CM}$",
    R"$\phi_2^{CM} \equiv \phi_{\pi^0}^{CM}$",
    R"$\phi_3^{CM} \equiv \phi_{p}^{CM}$",
]
theta_subtitles = [
    R"$\cos (\theta_{1}^{{12}}) \equiv \cos (\theta_{\eta}^{{\eta \pi^0}})$",
    R"$\cos (\theta_{2}^{{23}}) \equiv \cos (\theta_{\pi^0}^{{\pi^0 p}})$",
    R"$\cos (\theta_{3}^{{31}}) \equiv \cos (\theta_{p}^{{p \eta}})$",
]
phi_subtitles = [
    R"$\phi_1^{12} \equiv \phi_{\eta}^{{\eta \pi^0}}$",
    R"$\phi_2^{23} \equiv \phi_{\pi^0}^{{\pi^0 p}}$",
    R"$\phi_3^{31} \equiv \phi_{p}^{{p \eta}}$",
]
mass_subtitles = [
    R"$m_{12} \equiv m_{\eta \pi^0}$",
    R"$m_{23} \equiv m_{\pi^0 p}$",
    R"$m_{31} \equiv m_{p \eta}$",
]

fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)
for i, ax1 in enumerate(thetaCM_ax, 1):
    ax1.set_title(thetaCM_subtitles[i - 1])
    ax1.set_xticks([-1, 0, 1])
for i, ax2 in enumerate(phiCM_ax, 1):
    ax2.set_title(phiCM_subtitles[i - 1])
    ax2.set_xticks([-np.pi, 0, np.pi])
    ax2.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])

for i, ax3 in enumerate(theta_ax, 1):
    ax3.set_title(theta_subtitles[i - 1])
    ax3.set_xticks([-1, 0, 1])

for i, ax4 in enumerate(phi_ax, 1):
    ax4.set_title(phi_subtitles[i - 1])
    ax4.set_xticks([-np.pi, 0, np.pi])
    ax4.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])

for i, ax5 in enumerate(mass_ax, 1):
    ax5.set_title(mass_subtitles[i - 1])

thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


thetaCM_ax[0].hist(
    np.cos(theta1_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)

phiCM_ax[0].hist(
    phi1_CM_data,
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[1].hist(
    phi2_CM_data,
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[2].hist(
    phi3_CM_data,
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)

theta_ax[0].hist(
    np.cos(theta1_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_data),
    bins=100,
    label="data",
    density=True,
)

phi_ax[0].hist(
    phi1_data,
    bins=100,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=100,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=100,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()

fig.tight_layout()
plt.show()

## Fitting 

The initial guess of parameters before fittings.
We make a small offset for some default parameters, in an arbitrary choice of values, which are usually the case as initial guess before fitting.

In this particular situation, the parameters (to be fitted) are mass $M$ ($M_{12}$, $M_{23}$, and $M_{31}$) and decay width $\Gamma$ (($\Gamma_{12}$, $\Gamma_{23}$, and $\Gamma_{31}$)).
While the kinematics variables (Mandelstam variable $s$ ($s_{12}$, $s_{23}$, and $s_{31}$) and helicity angles $\theta$ (($\theta_{12}$, $\theta_{23}$, and $\theta_{31}$)) and $\phi$ (($\phi_{12}$, $\phi_{23}$, and $\phi_{31}$))) are used from phase space.

### Starting values for parameters

Firstly, have a look at the parameter defaults

In [ ]:
initial_parameters

Next, the new starting values of parameter is set by shifting a small value from the parameter defaults 

In [ ]:
new_initial_parameters = dict(
    M12=1.4,
    Gamma12=0.15,
    M23=1.59,
    Gamma23=0.05,
    M31=1.82,
    Gamma31=0.2,
    # a_minus2 is set to fixed and not go for fitting
    a_minus1=0.51,
    a_0=3.49,
    a_plus1=4.08,
    a_plus2=2.6,
    b_minus1=-1.2,
    b_0=4.04,
    b_plus1=0.41,
    c_0=2.66,
)

new_weight_BW_SH = BW_SH_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    **{
        **initial_parameters,
        **new_initial_parameters,
    },
)

In [ ]:
fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)
for i, ax1 in enumerate(thetaCM_ax, 1):
    ax1.set_title(thetaCM_subtitles[i - 1])
    ax1.set_xticks([-1, 0, 1])
for i, ax2 in enumerate(phiCM_ax, 1):
    ax2.set_title(phiCM_subtitles[i - 1])
    ax2.set_xticks([-np.pi, 0, np.pi])
    ax2.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(theta_subtitles[i - 1])
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(phi_subtitles[i - 1])
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(mass_subtitles[i - 1])


thetaCM_ax[0].hist(
    np.cos(theta1_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)

theta_ax[0].hist(
    np.cos(theta1_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_data),
    bins=100,
    label="data",
    density=True,
)

phiCM_ax[0].hist(
    phi1_CM_data,
    bins=50,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[1].hist(
    phi2_CM_data,
    bins=50,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[2].hist(
    phi3_CM_data,
    bins=50,
    label="data",
    density=True,
    alpha=0.5,
)
phi_ax[0].hist(
    phi1_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=50,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)

thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"CM angles, Helicity angles and invariant mass (Before fitting: starting parameters for BW $\times$ SH model)",
    y=1,
)
fig.tight_layout()
plt.show()

### Estimator and unbinned Log likelihood function

To perform a fit for parameters estimation by using unbinned log likelihood function:

In [ ]:
def unbinned_nll(
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
    a_minus2,
    a_minus1,
    a_0,
    a_plus1,
    a_plus2,
    b_minus1,
    b_0,
    b_plus1,
    c_0,
) -> float:
    phsp = (
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
    )
    parameter = dict(
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    )
    data = (
        p12_data.m2,
        p23_data.m2,
        p31_data.m2,
        phi1_data,
        theta1_data,
        phi2_data,
        theta2_data,
    )
    model_integral = BW_SH_model(*phsp, **parameter).mean()
    data_intensities = BW_SH_model(*data, **parameter)
    likelihoods = data_intensities / model_integral
    log_likelihood = np.log(likelihoods).sum()
    return -log_likelihood

:::{tip}
The phase space (phsp) is used for the normalization calculation for the unbinned Log likelihood function
:::

In [ ]:
%%time
unbinned_nll(
    **{
        **initial_parameters,
        **new_initial_parameters,
    },
)

### Optimizer

As a final step in this work, it is necessary to go for finding the parameter values that maximize this function, i.e., performing the optimization. 

This can be e.g. performed by `iminuit` package in python.

In [ ]:
from tqdm.auto import tqdm

In [ ]:
arg_names = tuple(new_initial_parameters)
progress_bar = None
global_function_calls = 0


def wrapped_estimator(*args: float) -> float:
    # minuit migrad expects positional-argument functions
    global progress_bar, global_function_calls
    if progress_bar is None:
        progress_bar = tqdm(desc="Running fit")
    progress_bar.update()
    global_function_calls += 1
    new_kwargs = {
        **initial_parameters,
        **dict(zip(arg_names, args)),
    }
    return unbinned_nll(**new_kwargs)

In [ ]:
import time

from iminuit import Minuit

optimizer = Minuit(
    wrapped_estimator,
    **new_initial_parameters,
    name=new_initial_parameters,
)
optimizer.errordef = Minuit.LIKELIHOOD
optimizer

In [ ]:
start_time = time.time()
optimizer.migrad()
end_time = time.time()
progress_bar.close()
progress_bar = (
    None  # Ensure to clean up by closing and removing the progress bar object
)

In [ ]:
duration = end_time - start_time
print(
    f"Optimization took {duration:.2f} seconds and {global_function_calls} function (steps/iterations) calls."
)

### Final fit result

In [ ]:
print("The result after optimization:")
optimizer.params

In [ ]:
optimized_parameters = {p.name: p.value for p in optimizer.params}
rounded_optimized_parameters = {p.name: round(p.value, 3) for p in optimizer.params}

parameters_df = pd.DataFrame({
    "Parameter": list(initial_parameters.keys()),
    "Default Value": list(initial_parameters.values()),
    "Starting Value": [new_initial_parameters.get(name) for name in initial_parameters],
    "Optimized Value": [
        rounded_optimized_parameters.get(name) for name in initial_parameters
    ],
})

print("Comparison of Default, Starting, and Optimized (Fitted) Parameters:")
parameters_df

In [ ]:
fitted_weight_BW_SH = BW_SH_model(
    s12_phsp,
    s23_phsp,
    s31_phsp,
    phi1_phsp,
    theta1_phsp,
    phi2_phsp,
    theta2_phsp,
    **{
        **initial_parameters,
        **optimized_parameters,
    },
)

In [ ]:
fig, (thetaCM_ax, phiCM_ax, theta_ax, phi_ax, mass_ax) = plt.subplots(
    figsize=(13, 18), ncols=3, nrows=5
)
for i, ax1 in enumerate(thetaCM_ax, 1):
    ax1.set_title(thetaCM_subtitles[i - 1])
    ax1.set_xticks([-1, 0, 1])
for i, ax2 in enumerate(phiCM_ax, 1):
    ax2.set_title(phiCM_subtitles[i - 1])
    ax2.set_xticks([-np.pi, 0, np.pi])
    ax2.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(theta_subtitles[i - 1])
    ax.set_xticks([-1, 0, 1])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(phi_subtitles[i - 1])
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(mass_subtitles[i - 1])


thetaCM_ax[0].hist(
    np.cos(theta1_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_data),
    bins=100,
    label="data",
    density=True,
    alpha=0.5,
)

theta_ax[0].hist(
    np.cos(theta1_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_data),
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_data),
    bins=100,
    label="data",
    density=True,
)

phiCM_ax[0].hist(
    phi1_CM_data,
    bins=50,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[1].hist(
    phi2_CM_data,
    bins=50,
    label="data",
    density=True,
    alpha=0.5,
)
phiCM_ax[2].hist(
    phi3_CM_data,
    bins=50,
    label="data",
    density=True,
    alpha=0.5,
)
phi_ax[0].hist(
    phi1_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=50,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)

thetaCM_ax[0].hist(
    np.cos(theta1_CM_phsp),
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[1].hist(
    np.cos(theta2_CM_phsp),
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
thetaCM_ax[2].hist(
    np.cos(theta3_CM_phsp),
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[0].hist(
    phi1_CM_phsp,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[1].hist(
    phi2_CM_phsp,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phiCM_ax[2].hist(
    phi3_CM_phsp,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

theta_ax[0].hist(
    np.cos(theta1_phsp),
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    np.cos(theta2_phsp),
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    np.cos(theta3_phsp),
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[0].hist(
    phi1_phsp,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2_phsp,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3_phsp,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[0].hist(
    p12_phsp.m,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23_phsp.m,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31_phsp.m,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

thetaCM_ax[0].legend()
phiCM_ax[0].legend()
thetaCM_ax[1].legend()
phiCM_ax[1].legend()
thetaCM_ax[2].legend()
phiCM_ax[2].legend()

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"CM angles, Helicity angles and invariant mass (After fitting: updated fitted parameters for BW $\times$ SH model)",
    y=1,
)
fig.tight_layout()
plt.show()

#### Dalitz plots of phsp, data, and fit result

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle(R"Dalitz Plots of Phase space, generated data")
hist2 = ax2.hist2d(s12_data, s23_data, bins=100, cmin=1e-6, density=True, cmap="jet")
ax2.set_xlabel(R"$m^2(\eta \pi)$")
ax2.set_title("Generated data")
ax2.set_ylabel(R"$m^2(\pi p)$")

hist1 = ax1.hist2d(s12_phsp, s23_phsp, bins=100, cmin=1e-2, density=True, cmap="jet")
ax1.set_title("Phase space")
ax1.set_xlabel(R"$m^2(\eta \pi)$")
ax1.set_ylabel(R"$m^2(\pi p)$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)

fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle(R"Dalitz Plots of parameter defaults and fitted parameters")

hist1 = ax1.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=BW_SH_model(
        s12_phsp,
        s23_phsp,
        s31_phsp,
        phi1_phsp,
        theta1_phsp,
        phi2_phsp,
        theta2_phsp,
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
        a_minus2=a_minus2,
        a_minus1=a_minus1,
        a_0=a_0,
        a_plus1=a_plus1,
        a_plus2=a_plus2,
        b_minus1=b_minus1,
        b_0=b_0,
        b_plus1=b_plus1,
        c_0=c_0,
    ),
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
)
ax1.set_title(R"Parameters defaults BW $\times$ SH model")
ax1.set_xlabel(R"$m^2(\eta \pi)$")
ax1.set_ylabel(R"$m^2(\pi p)$")

hist2 = ax2.hist2d(
    s12_phsp,
    s23_phsp,
    bins=100,
    weights=fitted_weight_BW_SH,
    cmin=1e-6,
    density=True,
    cmap="jet",
    vmax=0.15,
)
ax2.set_title(R"Fitted parameters BW $\times$ SH model")
ax2.set_xlabel(R"$m^2(\eta \pi)$")
ax2.set_ylabel(R"$m^2(\pi p)$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()